In [2]:
#import gensim.function

text=open('Стихи.txt','r').readlines()

In [3]:
text = [i.strip() for i in text]

In [4]:
text = [i for i in text if i != '' and 'Стихотворения' not in i]
text = text[:len(text)-243]
data=''.join(text)

In [5]:
len(data)

603368

In [6]:
data = data.replace('\xa0',' ')

for number in '123456789':
    data=data.replace(number,'0')
for english_letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
    data=data.replace(english_letter, '')
for english_letter in 'abcdefghijklmnopqrstuvwzyz':
    data=data.replace(english_letter, '')    

In [7]:
b=set(data)
c=dict()
for symbol in b:
    c[symbol] = data.count(symbol)
import operator
c1 = sorted(c.items(), key=operator.itemgetter(1))

In [8]:
Alphabet =''.join(c.keys())
Alphabet
#We don't delete any of the chars as now

'Ш?ъя: «Щы)ЖЧЭ\'>чщХ;…Ни&<жПьвз(ЙюКйпВОдАЗЦГ–Б0,шФр[—м_цнЮ#фМэх-]Д!Стб„г/УРxуТЛИ"л*кёса»ЕЯо.е'

In [9]:

def string_vectorizer(string, alphabet=Alphabet):
    vector = [[0 if char != letter else 1 for char in Alphabet] 
                  for letter in string]
    return vector
def vector_stringizer(vector, alphabet = Alphabet,return_first=True):
    answ=''
    if return_first:
        index = vector.argsort()[-1]
        answ+=Alphabet[index]
    else:
        index = vector.argsort()[-2]
        answ+=Alphabet[index]
    return answ

In [10]:

data_onehot = string_vectorizer(data)

In [11]:
import keras
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Flatten,Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
#from tqdm import tqdm
from scipy import sparse

C:\Users\DK\Anaconda3.1\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [11]:
window_len = 15
X_total,Y_total=[],[]
i=1000
X_total.append(data_onehot[i-window_len:i])
Y_total.append(data_onehot[i])

In [12]:
def make_dataset(data_onehot):
    X_total,Y_total=[],[]
    for i in range(window_len, len(data_onehot)):
        X_total.append(data_onehot[i-window_len:i])
        Y_total.append(data_onehot[i])
    return np.array(X_total), np.array(Y_total)
X_total, Y_total=make_dataset(data_onehot)

In [13]:
X_total.shape

(595977, 15, 91)

In [14]:
X_train,X_test,Y_train,Y_test = train_test_split(X_total, Y_total, test_size=0.15, random_state = 1)
del X_total
del  Y_total

In [12]:
from  keras.callbacks import EarlyStopping
callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='min')

In [ ]:
model = Sequential()
model.add(LSTM(100, input_shape=(window_len, len(Alphabet)), return_sequences=True))
model.add(Flatten())
model.add(Dense(len(Alphabet),activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam')
model.summary()
model.fit(X_train,Y_train, epochs=20,validation_data = (X_test, Y_test), batch_size=10, verbose=1,callbacks=[callback])

In [ ]:
model.save('Pushkin1.h5')  # creates a HDF5 file 'my_model.h5'
  # deletes the existing model

# returns a compiled model
# identical to the previous one
#model = load_model('Pushkin1.h5')

In [93]:
text = 'Уж небо осенью '
from copy import deepcopy
text1 = deepcopy(text)

Предсказываем наиболее вероятную букву

In [96]:
for i in range(1000):
    text_array = np.array(string_vectorizer(text1)).reshape(1,window_len,91)
    predicted_vector = model.predict(text_array)
    predicted_char = vector_stringizer(predicted_vector[0])
    text = text+predicted_char
    text1 = text1[1:]+predicted_char
    #if i%20==0:
        #print('Predicted '+str(i)+' letters ')
print(text)    

Уж небо осенью тобой.И вдруг и старик последний свободы не стала в силах восторгом забытый под стол мой друг, и сладостный поэт!То в сердце приветственный край сиял,Странал мой кругом,И постали в приятным столицы молодой,И сладостный покров и сладострастный под стол мой друг, и сладостный поэт!То в сердце приветственный край сиял,Странал мой кругом,И постали в приятным столицы молодой,И сладостный покров и сладострастный под стол мой друг, и сладостный поэт!То в сердце приветственный край сиял,Странал мой кругом,И постали в приятным столицы молодой,И сладостный покров и сладострастный под стол мой друг, и сладостный поэт!То в сердце приветственный край сиял,Странал мой кругом,И постали в приятным столицы молодой,И сладостный покров и сладострастный под стол мой друг, и сладостный поэт!То в сердце приветственный край сиял,Странал мой кругом,И постали в приятным столицы молодой,И сладостный покров и сладострастный под стол мой друг, и сладостный поэт!То в сердце приветственный край сиял,

In [99]:
text = 'Уж небо осенью '
text1 = deepcopy(text)

Предсказываем вторую по вероятности букву

In [100]:
for i in range(1000):
    text_array = np.array(string_vectorizer(text1)).reshape(1,window_len,91)
    predicted_vector = model.predict(text_array)
    predicted_char = vector_stringizer(predicted_vector[0], return_first=False)
    text = text+predicted_char
    text1 = text1[1:]+predicted_char
    #if i%20==0:
        #print('Predicted '+str(i)+' letters ')
print(text) 

Уж небо осенью свенцы, вечно тем прельем, прездучивых,Изманы,Помнешнному,Стоя поэзь тревеще небо в тем нахмерелых, скорее прелосая, всё,Всемурощей,Но,,Неталямь своей,Истиг в послуждалскою,Несестителях,Их,[час на стигах,Смолотен мирногде, никуплата нежиною твеяжь в придаю,Измалася,Дрогарайска прияжно мною послужны души,Или,Чадал из ни слог незносадей,Невинов днему.Явлеющей,Или,,С свердницами,С какновики,Возвождатей,Невиненно слогой,Или,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,

Предсказываем наиболее вероятную букву, но не всегда.

In [102]:
text = 'Уж небо осенью '
text1 = deepcopy(text)
for i in range(1000):
    text_array = np.array(string_vectorizer(text1)).reshape(1,window_len,91)
    predicted_vector = model.predict(text_array)
    if i%15!=0:
        predicted_char = vector_stringizer(predicted_vector[0], return_first=True)
    else:
        predicted_char = vector_stringizer(predicted_vector[0], return_first=False)
    text = text+predicted_char
    text1 = text1[1:]+predicted_char
    #if i%20==0:
        #print('Predicted '+str(i)+' letters ')
print(text) 

Уж небо осенью столь, и тому подъемлет он в сень на постали мой друг, и славы постали в прах веселых под шумит,И все дару под сень на свободы не стала,И сладостный простой пред тобы томный старик последний свобы в силой друг,В том за староских последний стал и сладость в полязу под себя возвышенный свободы,На моренья милый друг,Придать в свои под сень мой драгой старик постали в приходит,Он в своей прелестной полной старик последним простой мой долго страстной старик под сень на стал и старого столеты,В мечтанье старик и столковал он в пред ним столем постали в прах веселых под шумит,И все дару под сень на свободы не стала,И сладостный простой пред тобы томный старик последний свобы в силой друг,В том за староских последний стал и сладость в полязу под себя возвышенный свободы,На моренья милый друг,Придать в свои под сень мой драгой старик постали в приходит,Он в своей прелестной полной старик последним простой мой долго страстной старик под сень на стал и старого столеты,В мечтанье ст

Все равно циклится. А если выбирать вторую букву вместо первой не всегда,а более-менее рандомно?

In [106]:
text = 'Уж небо осенью '
text1 = deepcopy(text)
from numpy import random
random.seed(0)
for i in range(1000):
    text_array = np.array(string_vectorizer(text1)).reshape(1,window_len,91)
    predicted_vector = model.predict(text_array)
    j = random.randint(0,30)
    if j%25!=0:
        predicted_char = vector_stringizer(predicted_vector[0], return_first=True)
    else:
        predicted_char = vector_stringizer(predicted_vector[0], return_first=False)
    text = text+predicted_char
    text1 = text1[1:]+predicted_char
    #if i%20==0:
        #print('Predicted '+str(i)+' letters ')
print(text) 

Уж небо осенью тоба восторгом полетит, и странник мой,Как был под слезы просто полной старик послушный,Над собности подавал в приятной другием восторгом стоит простой мой друг, и сладостный, не страдал восторгом столом полетит,И в сердца и старик,Но ты, как девица страдала.Стал и сладостной постали в пред колени полный светил восторгом столетом заботы,На стану в молчанье, восторгом столом последний расстал и полнощный сводом.И странный, воскресный поэт,Но с подруги полной старик последний свободы не стали в полезный под сени, столицы стол,На полны на странного страдал, от бранный старик последний свои постали, не стройный красной?Ужель ужа ли в столе приведет.Посталься в другим столиться, в столе мой придет от приведеньем последний свободы не стала проститься на кругом полетит,И старик мой проститель полетели,И все не страда вознастить, постали в приятным столицы молитвый,В получил с невозок на странник старик, под скалоть, мой друг, издаливый свобы в сили в постеле под сень мира,И стр

Тоже бред, но по крайней мере не циклится. Уже лучше.

Дубль два. Учим более сложную модель, но с дропаутом. 

In [24]:
window_len = 15
model1 = Sequential()
model1.add(LSTM(120, input_shape=(window_len, len(Alphabet)), return_sequences=True))
model1.add(Flatten())
model1.add(Dropout(0.1))
model1.add(Dense(len(Alphabet),activation='softmax'))
model1.compile(loss='binary_crossentropy', optimizer='adam')
model1.summary()
model1.fit(X_train,Y_train, epochs=20,validation_data = (X_test, Y_test), batch_size=10, verbose=1,callbacks=[callback])


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 15, 120)           101760    
_________________________________________________________________
flatten_4 (Flatten)          (None, 1800)              0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 1800)              0         
_________________________________________________________________
dense_4 (Dense)              (None, 91)                163891    
Total params: 265,651
Trainable params: 265,651
Non-trainable params: 0
_________________________________________________________________
Train on 506580 samples, validate on 89397 samples
Epoch 1/20
506580/506580 [==============================] - 530s 1ms/step - loss: 0.0371 - val_loss: 0.0342
Epoch 2/20
506580/506580 [==============================] - 547s 1ms/step - loss: 0.0334 - val_loss: 0.0325
E

KeyboardInterrupt: 

Проверяем.

In [25]:
model1.save('Pushkin2.h5')

In [28]:
from copy import deepcopy
text = 'Уж небо осенью '
text1 = deepcopy(text)
from numpy import random
random.seed(0)
for i in range(1000):
    text_array = np.array(string_vectorizer(text1)).reshape(1,window_len,91)
    predicted_vector = model1.predict(text_array)
    j = random.randint(0,40)
    if j%35!=0:
        predicted_char = vector_stringizer(predicted_vector[0], return_first=True)
    else:
        predicted_char = vector_stringizer(predicted_vector[0], return_first=False)
    text = text+predicted_char
    text1 = text1[1:]+predicted_char
    #if i%20==0:
        #print('Predicted '+str(i)+' letters ')
print(text) 

Уж небо осенью слава,И всё по светлом полетал,И с под сенью, поступный возведь весь веселый взосленных,С первых лет под волновал своей положит,И с тобой пред ним странный свет под вечерный полон,В своих под сенью подобной полон воздохнет страна своему странный свет под вечерный полон вознеслись под себя возврата не смеря,Не прихот и в подарился.Но всё тебя не вознеслись под сенью:Но скоро ль на сверенный светел,Иль в сердце послушная под старик, в тебя в последний рад, с тобою, странной приходит в полуночи,И сладострастья в сердце пред ним своем под сенью подобной полон воздохнуть под сенью на свое подарит:Без под вечерней под сени,Свои пред тобою,И слезы с подобрать в последний разговор,И сладострастной струится в темной под своих день,И в сердце простой верный свет, —Не вознес и волны.Пред ним старый свое восторгов прихоти не приходит на берег восторгов прихоти не приходит на свое вечерней,И сладостно странный страстному странный старик,Под сенью воспоминание странный свой тебя в пос

Вроде, немного получше. А что если расширить выборку? Добавить к стихам поэмы?

In [2]:
from tqdm import tqdm
text=open('Стихи+Поэмы.txt','r').readlines()
text = [i for i in text if i != '' and 'Стихотворения' not in i]
text = [i.strip() for i in text]
text = text[:len(text)-243]
data=''.join(text)
data = data.replace('\xa0',' ')

for number in '123456789':
    data=data.replace(number,'0')
for english_letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
    data=data.replace(english_letter, '')
for english_letter in 'abcdefghijklmnopqrstuvwzyz':
    data=data.replace(english_letter, '')    
b=set(data)
c=dict()
for symbol in b:
    c[symbol] = data.count(symbol)
import operator
c1 = sorted(c.items(), key=operator.itemgetter(1))
Alphabet =''.join(c.keys())
Alphabet
#We don't delete any of the chars as now

data_onehot = string_vectorizer(data)
window_len = 15
X_total,Y_total=[],[]
def make_dataset(data_onehot):
    X_total,Y_total=[],[]
    for i in tqdm(range(window_len, len(data_onehot))):
        X_total.append(data_onehot[i-window_len:i])
        Y_total.append(data_onehot[i])
    return np.array(X_total), np.array(Y_total)
X_total, Y_total=make_dataset(data_onehot)
del tqdm

NameError: name 'string_vectorizer' is not defined

In [14]:
X_train,X_test,Y_train,Y_test = train_test_split(X_total, Y_total, test_size=0.15, random_state = 1)
del X_total
del  Y_total

In [15]:
import sys
del sys.modules["tqdm"]

In [16]:
tqdm

NameError: name 'tqdm' is not defined

In [19]:
window_len = 15
model2 = Sequential()
model2.add(LSTM(120, input_shape=(window_len, len(Alphabet)), return_sequences=True))
model2.add(Flatten())
model2.add(Dropout(0.1))
model2.add(Dense(len(Alphabet),activation='softmax'))
model2.compile(loss='binary_crossentropy', optimizer='adam')
model2.summary()
model2.fit(X_train,Y_train, epochs=20,validation_data = (X_test, Y_test), batch_size=10, verbose=2,callbacks=[callback])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_6 (LSTM)                (None, 15, 120)           103680    
_________________________________________________________________
flatten_6 (Flatten)          (None, 1800)              0         
_________________________________________________________________
dropout_6 (Dropout)          (None, 1800)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 95)                171095    
Total params: 274,775
Trainable params: 274,775
Non-trainable params: 0
_________________________________________________________________
Train on 906445 samples, validate on 159961 samples
Epoch 1/20
 - 891s - loss: 0.0342 - val_loss: 0.0314
Epoch 2/20
 - 923s - loss: 0.0309 - val_loss: 0.0298
Epoch 3/20
 - 855s - loss: 0.0297 - val_loss: 0.0293
Epoch 4/20
 - 844s - loss: 0.0291 - val_loss: 0.0289
Epoc

In [38]:
from copy import deepcopy
text = 'Уж небо осенью '
text1 = deepcopy(text)
from numpy import random
random.seed(90)
for i in range(2000):
    text_array = np.array(string_vectorizer(text1)).reshape(1,window_len,len(Alphabet))
    predicted_vector = model2.predict(text_array)
    j = random.randint(0,40)
    if j%35!=0:
        predicted_char = vector_stringizer(predicted_vector[0], return_first=True)
    else:
        predicted_char = vector_stringizer(predicted_vector[0], return_first=False)
    text = text+predicted_char
    text1 = text1[1:]+predicted_char
    #if i%20==0:
        #print('Predicted '+str(i)+' letters ')
print(text) 

Уж небо осенью долго в сердце простой не страшно в стране славы старины страны страны, друзья, приятной страны славы,Не стоят пред ним молодой, признаю страх на свете не смелой странный славы,Не достойный страны славы,На свет в страх не стал под сень моей друг, не стал под сень моей друг сердце волны своей свет и страшно в стране славы старины страны страны, другой приятный славы,Пред ним свободно в предан от своей сон он страх не стал под сень моей друг, не стал под сень моей друзья,Как славы славы славы, славы славы славы, славы славы не смелой,И все дружбе, под сень моей друг, славы бессмертный прихотит в нем сердце волны свед,То все друзья молодой,В сердце пред ним страх не стал последний свой долго в нем сестрица своих постели приветствовал он странный славы страны славы,Не подружками странный славы страны славы,Не стал под горами своей свет,То страх сердце полный страх, неверный страны славы,Не стал под городой.Он в старуха странный славы,Не старый страх не страшал он веселой при

А если всегда только самую вероятную букву?

In [35]:
from copy import deepcopy
text = 'Уж небо осенью '
text1 = deepcopy(text)
from numpy import random
random.seed(0)
for i in range(1000):
    text_array = np.array(string_vectorizer(text1)).reshape(1,window_len,len(Alphabet))
    predicted_vector = model2.predict(text_array)
    
    predicted_char = vector_stringizer(predicted_vector[0], return_first=True)
    text = text+predicted_char
    text1 = text1[1:]+predicted_char
    #if i%20==0:
        #print('Predicted '+str(i)+' letters ')
print(text) 

Уж небо осенью долго в сердце полный взор,И в стране славы старины страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны страны ст

Ничего хорошего. Хотелось бы помнить больше 15 символов, но вот беда - все в память не помещаются. А если fit_generator попробовать?


In [39]:
del  X_train, Y_train, X_test, Y_test

In [3]:
import keras
from sklearn.model_selection import train_test_split
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM,Flatten,Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
#from tqdm import tqdm
from scipy import sparse
from  keras.callbacks import EarlyStopping
callback = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='min')


C:\Users\DK\Anaconda3.1\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [4]:
from tqdm import tqdm
from numpy import random
text=open('Стихи+Поэмы.txt','r').readlines()
text = [i for i in text if i != '' and 'Стихотворения' not in i]
text = [i.strip() for i in text]
text = text[:len(text)-243]
data=''.join(text)
data = data.replace('\xa0',' ')

for number in '123456789':
    data=data.replace(number,'0')
for english_letter in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
    data=data.replace(english_letter, '')
for english_letter in 'abcdefghijklmnopqrstuvwzyz':
    data=data.replace(english_letter, '')    
b=set(data)
c=dict()
for symbol in b:
    c[symbol] = data.count(symbol)
import operator
c1 = sorted(c.items(), key=operator.itemgetter(1))
Alphabet =''.join(c.keys())
Alphabet
#We don't delete any of the chars as now

def string_vectorizer(string, alphabet=Alphabet):
    vector = [[0 if char != letter else 1 for char in Alphabet] 
                  for letter in string]
    return vector
def vector_stringizer(vector, alphabet = Alphabet,return_first=True):
    answ=''
    if return_first:
        index = vector.argsort()[-1]
        answ+=Alphabet[index]
    else:
        index = vector.argsort()[-2]
        answ+=Alphabet[index]
    return answ
data_onehot = string_vectorizer(data)


In [5]:
data_onehot=np.array(data_onehot)

In [105]:
batch_size=1500
window_len=35

In [106]:
s=np.arange(window_len, len(data_onehot))
random.seed(333)
train_set = np.random.choice(np.arange(window_len,len(data_onehot)-window_len), int(0.85*(len(data_onehot)-window_len)), replace=False)
test_set=np.array(list(set(np.arange(window_len,len(data_onehot)-window_len))-set(train_set)))

In [107]:
len(test_set)

159923

In [108]:
def batch_generator(train_set):
    global g,g0
    final_data=None
    final_label=None
    counter=0
    this_count=0
    i=0
    while True:
        data = data_onehot[train_set[i]-window_len:train_set[i],:]
        #print(data.shape)
        label = data_onehot[train_set[i],:]
        if counter==0:
            final_data=data.reshape(1,data.shape[0],data.shape[1])
            final_label=label.reshape(1,label.shape[0])
        else:
            data=data.reshape((1,data.shape[0],data.shape[1]))
            label=label.reshape((1,label.shape[0]))
            try:
                final_data=np.concatenate([final_data,data],axis=0)
            except:
                print(i)
                raise Exception(str(i)+' '+str(final_data.shape)+' '+str(data.shape))
            try:                                
                final_label=np.concatenate([final_label,label],axis=0)
            except:
                raise Exception(str(final_label.shape)+' '+str(label.shape))    

        counter+=1
        i=(i+1)%len(train_set)
        if counter==batch_size:
            if final_data.shape==(batch_size,window_len,len(Alphabet)) and final_label.shape==(batch_size,len(Alphabet)):
                yield (final_data,final_label)
                this_count+=1
                if this_count%15==0:
                    print('completed '+str(i*100/len(train_set))+' percent')
            else:
                print('ERROR')
                print(i)
                g=final_data,final_label
                raise Exception(str(g[0].shape)+' '+str(g[1].shape))
            final_data=None
            final_label=None
            counter=0

In [109]:
train_set = train_set[:len(train_set)-(len(train_set)%batch_size)]
test_set = test_set[:len(test_set)-(len(test_set)%batch_size)]

In [111]:
model2 = Sequential()
model2.add(LSTM(120, input_shape=(window_len, len(Alphabet)), return_sequences=True))
model2.add(Flatten())
model2.add(Dropout(0.1))
model2.add(Dense(len(Alphabet),activation='softmax'))
model2.compile(loss='binary_crossentropy', optimizer='adam')
model2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_14 (LSTM)               (None, 35, 120)           103680    
_________________________________________________________________
flatten_14 (Flatten)         (None, 4200)              0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 4200)              0         
_________________________________________________________________
dense_14 (Dense)             (None, 95)                399095    
Total params: 502,775
Trainable params: 502,775
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model2.fit_generator(
    batch_generator(train_set), epochs=20,
    steps_per_epoch = len(train_set)//batch_size,
    validation_data = batch_generator(test_set),
    validation_steps = len(test_set)//batch_size,
    verbose=2, callbacks=[callback])

Epoch 1/20
completed 2.4834437086092715 percent
completed 4.966887417218543 percent
completed 7.450331125827814 percent
completed 9.933774834437086 percent
completed 12.417218543046358 percent
completed 14.900662251655628 percent
completed 17.3841059602649 percent
completed 19.867549668874172 percent
completed 22.350993377483444 percent
completed 24.834437086092716 percent
completed 27.31788079470199 percent
completed 29.801324503311257 percent
completed 32.28476821192053 percent
completed 34.7682119205298 percent
completed 37.25165562913907 percent
completed 39.735099337748345 percent
completed 42.21854304635762 percent
completed 44.70198675496689 percent
completed 47.18543046357616 percent
completed 49.66887417218543 percent
completed 52.152317880794705 percent
completed 54.63576158940398 percent
completed 57.11920529801324 percent
completed 59.602649006622514 percent
completed 62.086092715231786 percent
completed 64.56953642384106 percent
completed 67.05298013245033 percent
complete

In [ ]:
len(test_set)//batch_size

In [ ]:
from copy import deepcopy
text = 'Уж небо осенью '
text1 = deepcopy(text)
from numpy import random
random.seed(90)
for i in range(2000):
    text_array = np.array(string_vectorizer(text1)).reshape(1,window_len,len(Alphabet))
    predicted_vector = model2.predict(text_array)
    j = random.randint(0,window_len)
    if j%35!=0:
        predicted_char = vector_stringizer(predicted_vector[0], return_first=True)
    else:
        predicted_char = vector_stringizer(predicted_vector[0], return_first=False)
    text = text+predicted_char
    text1 = text1[1:]+predicted_char
    #if i%20==0:
        #print('Predicted '+str(i)+' letters ')
print(text) 

In [ ]:
model2.save('Pushkin3.h5')

In [ ]:
print(Alphabet)